# Hacker News Project

In this project I am going to look at posts on the Hacker News website from 2016.  I'm interested in if there is a relation between the time a post is created and the number of comments the post received, and I'm also interested in if the Ask HN or Show HN posts receive more comments.

I'm going to first change the spreadsheet data into a list of lists and to also remove the data about any posts that did not recieve comments.

In [1]:
from csv import reader
open_file = open('HN_posts_year_to_Sep_26_2016.csv')
read_file = reader(open_file)
hn = list(read_file)

print(hn[:4])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']]


In [3]:
hn_1 = []
for row in hn:
    comments = row[4]
    if comments != '0':
        hn_1.append(row)
        
print("Length of Original Dataset", len(hn))
print("Length of New Dataset", len(hn_1))
print(hn_1[:4])

Length of Original Dataset 293120
Length of New Dataset 80402
[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12578975', 'Saving the Hassle of Shopping', 'https://blog.menswr.com/2016/09/07/whats-new-with-your-style-feed/', '1', '1', 'bdoux', '9/26/2016 3:13'], ['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'], ['12578822', 'Amazons Algorithms Dont Find You the Best Deals', 'https://www.technologyreview.com/s/602442/amazons-algorithms-dont-find-you-the-best-deals/', '1', '1', 'yarapavan', '9/26/2016 2:26']]


I am now going to separate the header row from all the data rows:

In [4]:
header = hn_1[0]
hn_1 = hn_1[1:]
print(header)
print(hn_1[:4])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12578975', 'Saving the Hassle of Shopping', 'https://blog.menswr.com/2016/09/07/whats-new-with-your-style-feed/', '1', '1', 'bdoux', '9/26/2016 3:13'], ['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'], ['12578822', 'Amazons Algorithms Dont Find You the Best Deals', 'https://www.technologyreview.com/s/602442/amazons-algorithms-dont-find-you-the-best-deals/', '1', '1', 'yarapavan', '9/26/2016 2:26'], ['12578694', 'Emergency dose of epinephrine that does not cost an arm and a leg', 'http://m.imgur.com/gallery/th6Ua', '2', '1', 'dredmorbius', '9/26/2016 1:54']]


In this next step I am going to look at how many posts started with Ask HN vs Show HN:

In [26]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn_1:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print('Number of Ask HN posts:', len(ask_posts))
print('Number of Show HN posts:', len(show_posts))
print('All other posts:', len(other_posts))

Number of Ask HN posts: 6911
Number of Show HN posts: 5059
All other posts: 68431


In [32]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

13.744175951381855
9.810832180272781
